# Winning Space Race with Data Science
### IBM Data Science Capstone Project

# Introduction

This is my data science project based on a case study given by IBM Data Science Professional Certificate course. In this project, I will use data science and machine learning thinking process and tools to solve this fictional real-world problem. In this report, I included following deliverables:
* Data Collection using API
* Data Wrangling using Pandas library
* Exploratory data analysis using Pandas and visualization
* Interactive visual analytics using Folium and Plotly Dash
* Predict analysis using classification models in Scikit-learn library

This report summary all my learning and hand-on labs from the IBM Data Science Capstone Project course. For the full detailed document, please visit [my github](https://github.com/supat-roong/IBM-Data-Science-Capstone-Project).

# Scenario

In this capstone, I will predict if the Falcon 9 first stage will land successfully. SpaceX advertises Falcon 9 rocket launches on its website with a cost of 62 million dollars; other providers cost upward of 165 million dollars each, much of the savings is because SpaceX can reuse the first stage. Therefore if we can determine if the first stage will land, we can determine the cost of a launch. This information can be used if an alternate company wants to bid against SpaceX for a rocket launch. 

In this project, I am a data scientist working for a new rocket company named `Space Y` that would like to compete with SpaceX. My job is to determine the price of each launch by gathering information about SpaceX and creating dashboards. I also determine if SpaceX will reuse the first stage by training a machine learning model and use public information to predict if SpaceX will reuse the first stage.



# Data collection
In this project, I will collect Space X data using the SpaceX API.

### Import Libraries and Define Auxiliary Functions

In [ ]:
import requests
import pandas as pd
import numpy as np
import datetime

# Setting this option will print all collumns of a dataframe
pd.set_option('display.max_columns', None)
# Setting this option will print all of the data in a feature
pd.set_option('display.max_colwidth', None)

In [ ]:
# Takes the dataset and uses the rocket column to call the API and append the data to the list
def getBoosterVersion(data):
    for x in data['rocket']:
        response = requests.get("https://api.spacexdata.com/v4/rockets/"+str(x)).json()
        BoosterVersion.append(response['name'])

# Takes the dataset and uses the launchpad column to call the API and append the data to the list
def getLaunchSite(data):
    for x in data['launchpad']:
        response = requests.get("https://api.spacexdata.com/v4/launchpads/"+str(x)).json()
        Longitude.append(response['longitude'])
        Latitude.append(response['latitude'])
        LaunchSite.append(response['name'])
        
# Takes the dataset and uses the payloads column to call the API and append the data to the lists
def getPayloadData(data):
    for load in data['payloads']:
        response = requests.get("https://api.spacexdata.com/v4/payloads/"+load).json()
        PayloadMass.append(response['mass_kg'])
        Orbit.append(response['orbit'])
        
# Takes the dataset and uses the cores column to call the API and append the data to the lists
def getCoreData(data):
    for core in data['cores']:
            if core['core'] != None:
                response = requests.get("https://api.spacexdata.com/v4/cores/"+core['core']).json()
                Block.append(response['block'])
                ReusedCount.append(response['reuse_count'])
                Serial.append(response['serial'])
            else:
                Block.append(None)
                ReusedCount.append(None)
                Serial.append(None)
            Outcome.append(str(core['landing_success'])+' '+str(core['landing_type']))
            Flights.append(core['flight'])
            GridFins.append(core['gridfins'])
            Reused.append(core['reused'])
            Legs.append(core['legs'])
            LandingPad.append(core['landpad'])

### Requesting rocket launch data from SpaceX API

In [ ]:
spacex_url="https://api.spacexdata.com/v4/launches/past"
response = requests.get(spacex_url)

# Use json_normalize meethod to convert the json result into a dataframe
data_API = pd.json_normalize(response.json())

# Get the head of the dataframe
data_API.head()

### Cleaning data

In [ ]:
# Lets take a subset of our dataframe keeping only the features we want and the flight number, and date_utc.
data_API = data_API[['rocket', 'payloads', 'launchpad', 'cores', 'flight_number', 'date_utc']]

# We will remove rows with multiple cores because those are falcon rockets with 2 extra rocket boosters and rows that have multiple payloads in a single rocket.
data_API = data_API[data_API['cores'].map(len)==1]
data_API = data_API[data_API['payloads'].map(len)==1]

# Since payloads and cores are lists of size 1 we will also extract the single value in the list and replace the feature.
data_API['cores'] = data_API['cores'].map(lambda x : x[0])
data_API['payloads'] = data_API['payloads'].map(lambda x : x[0])

# We also want to convert the date_utc to a datetime datatype and then extracting the date leaving the time
data_API['date'] = pd.to_datetime(data_API['date_utc']).dt.date

# Using the date we will restrict the dates of the launches
data_API = data_API[data_API['date'] <= datetime.date(2020, 11, 13)]

### Use auxiliary functions to store data in lists and create a new dataframe

In [ ]:
#Global variables 
BoosterVersion = []
PayloadMass = []
Orbit = []
LaunchSite = []
Outcome = []
Flights = []
GridFins = []
Reused = []
Legs = []
LandingPad = []
Block = []
ReusedCount = []
Serial = []
Longitude = []
Latitude = []

In [ ]:
# Call Auxiliary Functions
getBoosterVersion(data_API)
getLaunchSite(data_API)
getPayloadData(data_API)
getCoreData(data_API)

In [ ]:
# combine the columns into a dictionary
dict_API = {'FlightNumber': list(data_API['flight_number']),
'Date': list(data_API['date']),
'BoosterVersion':BoosterVersion,
'PayloadMass':PayloadMass,
'Orbit':Orbit,
'LaunchSite':LaunchSite,
'Outcome':Outcome,
'Flights':Flights,
'GridFins':GridFins,
'Reused':Reused,
'Legs':Legs,
'LandingPad':LandingPad,
'Block':Block,
'ReusedCount':ReusedCount,
'Serial':Serial,
'Longitude': Longitude,
'Latitude': Latitude}

# create a Pandas data frame
df_API = pd.DataFrame(dict_API)
df_API.head()

# Data wrangling

### Filter the dataframe to only include `Falcon 9` launches

In [ ]:
# keep only falcon9 launches
df = df_API[df_API['BoosterVersion']=='Falcon 9']

# reset the FlgihtNumber column
df.loc[:,'FlightNumber'] = list(range(1, df.shape[0]+1))
df.reset_index(drop=True)
df.head()

### Dealing with Missing Values

In [ ]:
# We can see below that some of the rows are missing values in our dataset
df.isnull().sum()

In [ ]:
# Replace missing values in PayloadMass column by its mean value
# Calculate the mean value of PayloadMass column
PayloadMass_mean = df['PayloadMass'].mean()
# Replace the np.nan values with its mean value
df['PayloadMass'].replace(np.nan, PayloadMass_mean, inplace = True) 

In [ ]:
# The number of missing values of the PayLoadMass column change to zero
df.isnull().sum()

### Create a landing outcome label from Outcome column

In [ ]:
# find all landing outcome
landing_outcomes = df['Outcome'].value_counts()
for i,outcome in enumerate(landing_outcomes.keys()):
    print(i,outcome)

In [ ]:
# create a set of outcomes where the second stage did not land successfully
bad_outcomes=set(landing_outcomes.keys()[[1,3,5,6,7]])

In [ ]:
# identify good and bad outcome
# landing_class = 0 if bad_outcome
# landing_class = 1 otherwise
landing_class = []
for outcome in df['Outcome']:
    if outcome in bad_outcomes:
        landing_class.append(0)
    else:
        landing_class.append(1)

In [ ]:
# add to dataframe
df['Class']=landing_class
df.head()

### Identify data type of each column

In [ ]:
df.dtypes

# Exploratory data analysis

## Pandas

### Calculate the number of launches on each site

The data contains several Space X  launch facilities: <a href='https://en.wikipedia.org/wiki/List_of_Cape_Canaveral_and_Merritt_Island_launch_sites?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2022-01-01'>Cape Canaveral Space</a> Launch Complex 40  <b>VAFB SLC 4E </b> , Vandenberg Air Force Base Space Launch Complex 4E <b>(SLC-4E)</b>, Kennedy Space Center Launch Complex 39A <b>KSC LC 39A </b>.The location of each Launch Is placed in the column <code>LaunchSite</code>

In [ ]:
df['LaunchSite'].value_counts()

### Calculate the number and occurrence of each orbit
Here are some common orbit types:
*   <b>LEO</b>: Low Earth orbit (LEO)is an Earth-centred orbit with an altitude of 2,000 km (1,200 mi) or less (approximately one-third of the radius of Earth),\[1] or with at least 11.25 periods per day (an orbital period of 128 minutes or less) and an eccentricity less than 0.25.\[2] Most of the manmade objects in outer space are in LEO <a href='https://en.wikipedia.org/wiki/Low_Earth_orbit?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2022-01-01'>\[1]</a>.

*   <b>VLEO</b>: Very Low Earth Orbits (VLEO) can be defined as the orbits with a mean altitude below 450 km. Operating in these orbits can provide a number of benefits to Earth observation spacecraft as the spacecraft operates closer to the observation<a href='https://www.researchgate.net/publication/271499606_Very_Low_Earth_Orbit_mission_concepts_for_Earth_Observation_Benefits_and_challenges?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2022-01-01'>\[2]</a>.

*   <b>GTO</b> A geosynchronous orbit is a high Earth orbit that allows satellites to match Earth's rotation. Located at 22,236 miles (35,786 kilometers) above Earth's equator, this position is a valuable spot for monitoring weather, communications and surveillance. Because the satellite orbits at the same speed that the Earth is turning, the satellite seems to stay in place over a single longitude, though it may drift north to south,” NASA wrote on its Earth Observatory website <a  href="https://www.space.com/29222-geosynchronous-orbit.html?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2022-01-01" >\[3] </a>.

*   <b>SSO (or SO)</b>: It is a Sun-synchronous orbit  also called a heliosynchronous orbit is a nearly polar orbit around a planet, in which the satellite passes over any given point of the planet's surface at the same local mean solar time <a href="https://en.wikipedia.org/wiki/Sun-synchronous_orbit?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2022-01-01">\[4] <a>.

*   <b>ES-L1 </b>:At the Lagrange points the gravitational forces of the two large bodies cancel out in such a way that a small object placed in orbit there is in equilibrium relative to the center of mass of the large bodies. L1 is one such point between the sun and the earth <a href="https://en.wikipedia.org/wiki/Lagrange_point?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2022-01-01#L1_point">\[5]</a> .

*   <b>HEO</b> A highly elliptical orbit, is an elliptic orbit with high eccentricity, usually referring to one around Earth <a href="https://en.wikipedia.org/wiki/Highly_elliptical_orbit?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2022-01-01">\[6]</a>.

*   <b> ISS </b> A modular space station (habitable artificial satellite) in low Earth orbit. It is a multinational collaborative project between five participating space agencies: NASA (United States), Roscosmos (Russia), JAXA (Japan), ESA (Europe), and CSA (Canada)<a href="https://en.wikipedia.org/wiki/International_Space_Station?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2022-01-01"> \[7] </a>

*   <b> MEO </b> Geocentric orbits ranging in altitude from 2,000 km (1,200 mi) to just below geosynchronous orbit at 35,786 kilometers (22,236 mi). Also known as an intermediate circular orbit. These are "most commonly at 20,200 kilometers (12,600 mi), or 20,650 kilometers (12,830 mi), with an orbital period of 12 hours <a href="https://en.wikipedia.org/wiki/List_of_orbits?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2022-01-01"> \[8] </a>

*   <b> HEO </b> Geocentric orbits above the altitude of geosynchronous orbit (35,786 km or 22,236 mi) <a href="https://en.wikipedia.org/wiki/List_of_orbits?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2022-01-01"> \[9] </a>

*   <b> GEO </b> It is a circular geosynchronous orbit 35,786 kilometres (22,236 miles) above Earth's equator and following the direction of Earth's rotation <a href="https://en.wikipedia.org/wiki/Geostationary_orbit?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2022-01-01"> \[10] </a>

*   <b> PO </b> It is one type of satellites in which a satellite passes above or nearly above both poles of the body being orbited (usually a planet such as the Earth <a href="https://en.wikipedia.org/wiki/Polar_orbit?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2022-01-01"> \[11] </a>

![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/Orbits.png)

In [ ]:
df['Orbit'].value_counts()

### Find the landing success rate

In [ ]:
df["Class"].mean()

## Visualization

### Import visualization library

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

### Visualize the relationship between Payload and Launch Site
Firstly, we want to observe if there is any relationship between launch sites and their payload mass.

In [ ]:
sns.catplot(y="LaunchSite", x="PayloadMass", hue="Class", data=df, aspect = 5)
plt.xlabel("Pay load Mass (kg)",fontsize=20)
plt.ylabel("Launch Site",fontsize=20)
plt.show()

From Payload Vs. Launch Site scatter point chart, we can see that for the VAFB-SLC  launchsite there are no  rockets  launched for  heavypayload mass(greater than 10000).

### Visualize the relationship between success rate of each orbit type
Next, we want to visually check if there are any relationship between success rate and orbit type using bar chart.

In [ ]:
sns.barplot(y="Class", x="Orbit", data=df[['Orbit', 'Class']].groupby(by='Orbit').mean().reset_index())
plt.xlabel("Orbit",fontsize=20)
plt.ylabel("Success Rate",fontsize=20)
plt.show()

We can see that Orbit ES-L1, GEO, HEO, SSO have the best success rate. However, when consider the number of launches to each orbit, the Orbit ES-L1, GEO, HEO, SSO have only 1 launch, so its does not have much integrity. Therefore, after excluding those orbits, the VLEO orbit with the total of 14 launches have the highest success rate.

### Visualize the relationship between Payload and Orbit type
Then, we can plot the Payload vs. Orbit scatter point charts to reveal the relationship between Payload and Orbit type.

In [ ]:
sns.catplot(y="Orbit", x="PayloadMass", hue="Class", data=df, aspect = 5)
plt.xlabel("Pay load Mass (KG)",fontsize=20)
plt.ylabel("Orbit",fontsize=20)
plt.show()

With heavy payloads the successful landing or positive landing rate are more for Polar, LEO and ISS.

However for GTO we cannot distinguish this well as both positive landing rate and negative landing(unsuccessful mission) are both there here.


### Visualize the launch success yearly trend
Lastly, we want to visually check if there are any launch success yearly trend using line plot.

In [ ]:
# A function to Extract years from the date 
def Extract_year(date):
    for i in df["Date"]:
        year.append(int(i.strftime("%Y")))
    return year

In [ ]:
year =[]
df['Year'] = Extract_year(df)
sns.regplot(y="Class", x="Year", data=df[['Year', 'Class']].groupby('Year').mean().reset_index())
plt.xlabel("Year",fontsize=20)
plt.ylabel("Succsee Rate",fontsize=20)
plt.show()

As we can see above, the success rate tend to improve as the year pass.

# Interactive visual analytics

## Folium Map

### Import folium library

In [ ]:
import folium
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium DivIcon plugin
from folium.features import DivIcon

### Data wrangling

In [ ]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
df_folium = df[['LaunchSite', 'Latitude', 'Longitude', 'Class']]
df_launch_sites = df_folium.groupby(['LaunchSite'], as_index=False).first()
df_launch_sites = df_launch_sites[['LaunchSite', 'Latitude', 'Longitude']]
df_launch_sites

Above coordinates are just plain numbers that can not give any intuitive insights about where are those launch sites. 

Hence, let's visualize those locations by pinning them on a map.

### Mark all launch sites on a map

In [ ]:
# Initial the map
site_map = folium.Map(location = [29.559684888503615, -95.0830971930759],
                      zoom_start = 4,
                      max_bounds=True)

In [ ]:
# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label
for i, LaunchSite in enumerate(df_launch_sites['LaunchSite']):
    
    # Create a orange circle at each luanch station coordinate with a popup label showing its name
    
    circle = folium.Circle(df_launch_sites.loc[i, 'Latitude':'Longitude'].to_list(), radius=100, color='#d35400', fill=True).add_child(folium.Popup(LaunchSite))
    site_map.add_child(circle)
    # Create a orange circle at each luanch station coordinate with a icon showing its name
    marker = folium.map.Marker(
        df_launch_sites.loc[i, 'Latitude':'Longitude'].to_list(),
        # Create an icon as a text label
        icon=DivIcon(
            icon_size=(20,20),
            icon_anchor=(0,0),
            html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % LaunchSite,
            )
        )
    site_map.add_child(marker) 
site_map

After exploring the map by zoom-in/out the marked areas, we can see that
*   All launch sites in proximity to the Equator line
*   All launch sites in very close proximity to the coast

### Mark the success/failed launches for each site on the map
Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Since a launch only happens in one of the three launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.


In [ ]:
# Function to assign color to launch outcome
def assign_marker_color(launch_outcome):
    if launch_outcome == 1:
        return 'green'
    else:
        return 'red'

# Assign marker color for each row in dataframe
df_folium['marker_color'] = df_folium['Class'].apply(assign_marker_color)

In [ ]:
# Create marker cluster object
marker_cluster = MarkerCluster()

# Add marker_cluster to current site_map
site_map.add_child(marker_cluster)

# create a Marker object with its coordinate 
# and customize the Marker's icon property to indicate if this launch was successed or failed
for index, row in df_folium.iterrows():
    # Create and add a Marker cluster to the site map
    marker = folium.map.Marker([row['Latitude'], row['Longitude']]
        ,
        # Create an icon as a text label
        icon=folium.Icon(color='white', icon_color=row['marker_color'])
                              )
    marker_cluster.add_child(marker)

In [ ]:
# Set the location and zoom position to Launch Site CCSFS SLC 40
# You can click the marker cluster (Yellow) to find all launch results
location_CCSFS_SLC_40 = [28.561857, -80.577366]
bound = [0.001, 0.001]
sw = list(np.subtract(location_CCSFS_SLC_40, bound))
ne = list(np.add(location_CCSFS_SLC_40, bound))
site_map.fit_bounds([sw, ne])
site_map

From the color-labeled markers in marker clusters, you should be able to easily identify which launch sites have relatively high success rates.

## Plotly Dashboard

Since Dash app require a local host, which kaggle does not support, please visit my [github](https://github.com/supat-roong/IBM-Data-Science-Capstone-Project/blob/8057543ff91295026e7451afe3197ce1ead97e25/7.%20Dashboard%20Application%20with%20Plotly%20Dash.ipynb) for the full dashboard code. 

Here is the example of my Plotly Dash dashboard. This dashboard has 4 components including dropdown menu for selecting launch sites, pie chart to visualize success rate in each launch site, slider to select payload range, and scatter plot plot to visualize relationship launch site, payload, and booster version.

![Dashboard Example](https://github.com/supat-roong/IBM-Data-Science-Capstone-Project/blob/main/Dashboard%20Example.png?raw=true)

# Predict analysis

We will determine if the first stage rocket will land using predictive analysis by classification models.

### Import Libraries and Define Auxiliary Functions

In [ ]:
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
# This function is to plot the confusion matrix
def plot_confusion_matrix(y,y_predict):
    "this function plots the confusion matrix"
    from sklearn.metrics import confusion_matrix

    cm = confusion_matrix(y, y_predict)
    ax= plt.subplot()
    sns.heatmap(cm, annot=True, ax = ax); #annot=True to annotate cells
    ax.set_xlabel('Predicted labels')
    ax.set_ylabel('True labels')
    ax.set_title('Confusion Matrix'); 
    ax.xaxis.set_ticklabels(['did not land', 'land']); ax.yaxis.set_ticklabels(['did not land', 'landed'])

### Features Selection
From the exploratory data analysis, we obtain some preliminary insights about how each important variable would affect the success rate, we will select the features that will be used in success prediction.

In [ ]:
features = df[['FlightNumber', 'PayloadMass', 'Orbit', 'LaunchSite', 'Flights', 'GridFins', 'Reused', 'Legs', 'LandingPad', 'Block', 'ReusedCount', 'Serial']]
features.head()

#### Create dummy variables to categorical columns
Use the function <code>get_dummies</code> and <code>features</code> dataframe to apply OneHotEncoder to the column <code>Orbits</code>, <code>LaunchSite</code>, <code>LandingPad</code>, and <code>Serial</code>. Assign the value to the variable <code>features_one_hot</code>, display the results using the method head. 

In [ ]:
features_one_hot = pd.get_dummies(features[['Orbit', 'LaunchSite', 'LandingPad', 'Serial']])
features_one_hot.head()

#### Merge dummy variable to the feature

In [ ]:
X = features.merge(features_one_hot, left_index=True, right_index=True)
X.head()

#### Drop duplicate feature

In [ ]:
X.drop(['Orbit', 'LaunchSite', 'LandingPad', 'Serial'], axis=1, inplace=True)

#### Cast all numeric columns to `float64`

In [ ]:
X.astype('float64')

#### Select target variable
The target variable is `Class` column from the `df` dataframe

In [ ]:
Y = df['Class'].to_numpy()

### Preprocessing dataset
#### Standardize the data in <code>X</code>

In [ ]:
scaler = preprocessing.StandardScaler()
scaler.fit(X)
X=scaler.transform(X)

#### split the data `X` and `Y` into training and test data
For the consistency of this report, I set `random_state=2`

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=2)

we can see we only have 18 test samples.

In [ ]:
Y_test.shape

### Model training
#### Logistic Regression
Create a logistic regression object  then create a  GridSearchCV object  <code>logreg_cv</code> with cv = 10.  Fit the object to find the best parameters from the dictionary <code>parameters</code>.

In [ ]:
# parameters for logistic regression model
parameters ={'C':[0.01,0.1,1],
             'penalty':['l2'],
             'solver':['lbfgs']}

# create model and tune hyperparameter
lr=LogisticRegression()
logreg_cv = GridSearchCV(lr, parameters, cv=10)
logreg_cv.fit(X_train, Y_train)
logreg_cv_score = logreg_cv.score(X_test, Y_test)
print("Logistic Regression model")
print("tuned hypererparameters :(best parameters) ",logreg_cv.best_params_)
print("accuracy (train data):",logreg_cv.best_score_)
print("accuracy (test data):",logreg_cv_score)

Then, create a confusion matrix of logistic regression model

In [ ]:
# create confusion matrix
yhat=logreg_cv.predict(X_test)
plot_confusion_matrix(Y_test,yhat)

From the above result, we see that logistic regression model can distinguish between the different classes with the accuracy of 83.33%.  However, the major problem is false positives.

#### Support Vector Machine (SVM)
Create a support vector machine object then  create a  <code>GridSearchCV</code> object  <code>svm_cv</code> with cv - 10.  Fit the object to find the best parameters from the dictionary <code>parameters</code>.


In [ ]:
# parameters for SVM model
parameters = {'kernel':('linear', 'rbf','poly','rbf', 'sigmoid'),
              'C': np.logspace(-3, 3, 5),
              'gamma':np.logspace(-3, 3, 5)}

# create model and tune hyperparameter
svm = SVC()
svm_cv = GridSearchCV(svm, parameters, cv=10)
svm_cv.fit(X_train, Y_train)
svm_cv_score = svm_cv.score(X_test, Y_test)
print("Support Vector Machine model")
print("tuned hyperparameters :(best parameters) ",svm_cv.best_params_)
print("accuracy (train data):",svm_cv.best_score_)
print("accuracy (test data):",svm_cv_score)

Then, create a confusion matrix of SVM model

In [ ]:
# create confusion matrix
yhat=svm_cv.predict(X_test)
plot_confusion_matrix(Y_test,yhat)

From the above result, we see that SVM model can distinguish between the different classes with the accuracy of 83.33%.  However, the major problem is false positives.

#### Decision Tree Classifier
Create a decision tree classifier object then  create a  <code>GridSearchCV</code> object  <code>tree_cv</code> with cv = 10.  Fit the object to find the best parameters from the dictionary <code>parameters</code>.

In [ ]:
# parameters for decision tree classifier model
parameters = {'criterion': ['gini', 'entropy'],
     'splitter': ['best', 'random'],
     'max_depth': [2*n for n in range(1,10)],
     'max_features': ['auto', 'sqrt'],
     'min_samples_leaf': [1, 2, 4],
     'min_samples_split': [2, 5, 10]}

# create model and tune hyperparameter
tree = DecisionTreeClassifier()
tree_cv = GridSearchCV(tree, parameters, cv=10)
tree_cv.fit(X_train, Y_train)
tree_cv_score = tree_cv.score(X_test, Y_test)
print("Decision Tree Classifier model")
print("tuned hpyerparameters :(best parameters) ",tree_cv.best_params_)
print("accuracy (train data):",tree_cv.best_score_)
print("accuracy (test data):",tree_cv_score)

Then, create a confusion matrix of decision tree classifier model

In [ ]:
# create confusion matrix
yhat = tree_cv.predict(X_test)
plot_confusion_matrix(Y_test,yhat)

From the above result, we see that decision tree classifier model can distinguish between the different classes with the accuracy of 94.44%.  However, the major problem is false positives.

#### K Nearest Neighbors
Create a k nearest neighbors object then  create a  <code>GridSearchCV</code> object  <code>knn_cv</code> with cv = 10.  Fit the object to find the best parameters from the dictionary <code>parameters</code>.

In [ ]:
# parameters for k nearest neighbors model
parameters = {'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
              'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
              'p': [1,2]}

# create model and tune hyperparameter
KNN = KNeighborsClassifier()
knn_cv = GridSearchCV(KNN, parameters, cv=10)
knn_cv.fit(X_train, Y_train)
knn_cv_score = knn_cv.score(X_test, Y_test)
print("K Nearest Neighbors model")
print("tuned hpyerparameters :(best parameters) ",knn_cv.best_params_)
print("accuracy (train data):",knn_cv.best_score_)
print("accuracy (test data):",knn_cv_score)

Then, create a confusion matrix of k nearest neighbors model

In [ ]:
# create confusion matrix
yhat = knn_cv.predict(X_test)
plot_confusion_matrix(Y_test,yhat)

From the above result, we see that k nearest neighbors model can distinguish between the different classes with the accuracy of 77.78%.  However, the major problem is false positives.

### Compare model performance
Compare each model accuracy on test data using dataframe and bar chart.

In [ ]:
# model accuracy dataframe
model_dict = {"Model": ['Logistic Regression', 'Support Vector Machine', 'Decision Tree Classifier', 'K Nearest Neighbors'], 
              "Accuracy": [logreg_cv_score, svm_cv_score, tree_cv_score, knn_cv_score]}
model = pd.DataFrame(model_dict)
model

In [ ]:
# model accuracy comparison bar chart
sns.barplot(y="Accuracy", x="Model", data=model)
plt.xticks(rotation=45)
plt.xlabel("Model",fontsize=20)
plt.ylabel("Accuracy",fontsize=20)
plt.show()

Therefore, we can see that the best classification model is Decision Tree Classifier model with accuracy of 88.89%

# Conclusion

From the above report, I summarized this project deliverables in below.
* Data Collection using API
    * Collected SpaceX rocket launches data using SpaceX API and Auxiliary Functions
* Data Wrangling using Pandas library
    * Filtered to only include `Falcon 9` launches
    * Replaced missing values in `PayloadMass` column by its mean value
    * Created a landing outcome label from `Outcome` column
* Exploratory data analysis using Pandas and visualization
    * Calculated the number of launches on each feature
    * Calculated landing success rate
    * Identified trend and correlation using visualization
* Interactive visual analytics using Folium and Plotly Dash
    * Created Folium map with launch sites location and visualized their success rate using marker cluster objects
    * Created interactive dashboard using Plotly Dash including input objects (dropdown menu, slider) and output objects (visualization charts)
* Predict analysis using classification models in Scikit-learn library
    * Preprocessed data
        * One hot encoding for category data
        * Normalized data using standard scaler
        * Splitted train and test data
    * Trained and evaluated model
        * Trained classification models in Scikit-learn library
        * Tuned hyperparameters using `GridSearchCV` objects
        * Calculated each model performance
        * Visualized each model performance usig confusion matrix
        * Compared each model performance using table and charts

That is the end of my report, thank you for your reading.